In [8]:
import pickle
path = "C:/Users/user/Desktop/MLGame/games/pingpong/log/t (1).pickle"
file = open(path, "rb")
data = pickle.load(file)
info1 = data['ml_1P']['scene_info']
game_command1 = data['ml_1P']['command']
game_command2 = data['ml_2P']['command']
file.close()


for i in range(2, 243):
    path = "C:/Users/user/Desktop/MLGame/games/pingpong/log/t (" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    info1 = info1 + data['ml_1P']['scene_info']
    game_command1 = game_command1 + data['ml_1P']['command']
    game_command2 = game_command2 + data['ml_2P']['command']
    file.close()

for i in range(1, 1940):
    path = "C:/Users/user/Desktop/MLGame/games/pingpong/log/feature (" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    info1 = info1 + data['ml_1P']['scene_info']
    game_command1 = game_command1 + data['ml_1P']['command']
    game_command2 = game_command2 + data['ml_2P']['command']
    file.close()


In [9]:
# 提取特徵
bx = []
by = []
bsx = []
bsy = []
s = []
p1 = []
p2 = []
b = []
bs = []
command1 = []
command2 = []

c = False
for i in range(1, len(info1)):
    scene_info = info1[i]
    c1 = game_command1[i]
    c2 = game_command2[i]
    status = scene_info['status']
    if status != 'GAME_ALIVE':
        c1 = False
        c2 = False
        continue
    else:
        if c1 == 'SERVE_TO_LEFT' or c1 == 'SERVE_TO_RIGHT' or c2 == 'SERVE_TO_LEFT' or c2 == 'SERVE_TO_RIGHT':
            c = True
            continue
    if c:   
        ball_x = scene_info['ball'][0]
        ball_y = scene_info['ball'][1]
        ball_speed_x = scene_info["ball_speed"][0]
        ball_speed_y = scene_info["ball_speed"][1]
        slope = ball_speed_y/ball_speed_x if ball_speed_x != 0 else 0
        platform1 = scene_info['platform_1P'][0]
        platform2 = scene_info['platform_2P'][0]
        if "blocker" in scene_info:
            blocker = scene_info["blocker"][0]
            blocker_s = 5 if scene_info["blocker"][0] - info1[i-1]["blocker"][0] > 0 else -5
            if slope != 0:
                bx.append(ball_x)
                by.append(ball_y)
                bsx.append(ball_speed_x)
                bsy.append(ball_speed_y)
                s.append(slope)
                p1.append(platform1)
                p2.append(platform2)
                b.append(blocker)
                bs.append(blocker_s)
                if c1 == "NONE":
                    command1.append(0)
                elif c1 == "MOVE_LEFT":
                    command1.append(-1)
                elif c1 == "MOVE_RIGHT":
                    command1.append(1)
                if c2 == "NONE":
                    command2.append(0)
                elif c2 == "MOVE_LEFT":
                    command2.append(-1)
                elif c2 == "MOVE_RIGHT":
                    command2.append(1)
        else:
            if slope != 0:
                bx.append(ball_x)
                by.append(ball_y)
                bsx.append(ball_speed_x)
                bsy.append(ball_speed_y)
                s.append(slope)
                p1.append(platform1)
                p2.append(platform2)
                b.append(-1)
                bs.append(-1)
                if c1 == "NONE":
                    command1.append(0)
                elif c1 == "MOVE_LEFT":
                    command1.append(-1)
                elif c1 == "MOVE_RIGHT":
                    command1.append(1)
                if c2 == "NONE":
                    command2.append(0)
                elif c2 == "MOVE_LEFT":
                    command2.append(-1)
                elif c2 == "MOVE_RIGHT":
                    command2.append(1)
print(len(p1))
print(len(p2))
print(len(command1))
print(len(command2))

3511726
3511726
3511726
3511726


In [10]:
import numpy as np

numpy_data1 = np.array([bx, by, bsx, bsy, s, p1, b, bs])
numpy_data2 = np.array([bx, by, bsx, bsy, s, p2, b, bs])
X1 = np.transpose(numpy_data1) 
y1 = command1
X2 = np.transpose(numpy_data2) 
y2 = command2

In [ ]:
# train data KNN
from sklearn.neighbors import KNeighborsClassifier

model1 = KNeighborsClassifier(n_neighbors=2)
model1.fit(X1, y1)
print(model1.score(X1, y1))
model2 = KNeighborsClassifier(n_neighbors=2)
model2.fit(X2, y2)
print(model2.score(X2, y2))

In [ ]:
with open('C:/Users/user/Desktop/MLGame/games/pingpong/ml/model_1P_knn.pickle', 'wb') as p1:
    pickle.dump(model1, p1)
with open('C:/Users/user/Desktop/MLGame/games/pingpong/ml/model_2P_knn.pickle', 'wb') as p2:
    pickle.dump(model2, p2)

In [11]:
"""from sklearn.ensemble import RandomForestClassifier
rfc1 = RandomForestClassifier(max_depth = 20)
rfc1.fit(X1,y1)
rfc2 = RandomForestClassifier(max_depth = 20)
rfc2.fit(X2,y2)"""

RandomForestClassifier(max_depth=20)

In [12]:
"""with open('C:/Users/user/Desktop/MLGame/games/pingpong/ml/model_1P_rfc.pickle', 'wb') as p1:
    pickle.dump(rfc1, p1)
with open('C:/Users/user/Desktop/MLGame/games/pingpong/ml/model_2P_rfc.pickle', 'wb') as p2:
    pickle.dump(rfc2, p2)"""

In [ ]:
"""# train data SVM classification
from sklearn import svm
#ppc1 = svm.SVC(gamma = 0.001, decision_function_shape = 'ovo')
#ppc1.fit(X1,y1)

#ppc2 = svm.SVC(gamma = 0.001, decision_function_shape = 'ovo')
#ppc2.fit(X2,y2)

ppc1 = svm.SVC(gamma = 0.001, decision_function_shape = 'ovr')
ppc1.fit(X1,y1)

ppc2 = svm.SVC(gamma = 0.001, decision_function_shape = 'ovr')
ppc2.fit(X2,y2)"""

In [ ]:
"""with open('C:/Users/user/Desktop/MLGame/games/pingpong/ml/model_1P_svm.pickle', 'wb') as p1:
    pickle.dump(ppc1, p1)
with open('C:/Users/user/Desktop/MLGame/games/pingpong/ml/model_2P_svm.pickle', 'wb') as p2:
    pickle.dump(ppc2, p2)"""